# Fraud detection - Data Understanding

I have used the github copilot exentsion to write the code, it uses my reposetories to predict what i want to write based on the first few words of code. 

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

💬 **Interpret this output:** What does it tell you about the data?

In [ ]:
# Load dataset
url = 'https://raw.githubusercontent.com/Patrick0481/Data-science-for-business/refs/heads/main/Week2_Fraud_Detection.csv?token=GHSAT0AAAAAADMGVMALDQKAOVOYMJL2ZVH42G3Z4RQ'
df = pd.read_csv(url)
print(df.head())
print(df.info())

#This shows the different columns and their data types

💬 **Interpret this output:** What does it tell you about the data?

In [ ]:
# Check for missing values and duplicates
print(df.isnull().sum())
print(df.duplicated().sum())

#There are no missing values or duplicates in the dataset

💬 **Interpret this output:** What does it tell you about the data?

In [ ]:
# Summary stats
print(df.describe())

#Rows: 300
#Amount mean ≈ 97.28, median ≈ 71.68, max 461.06
#Fraudulent: 26 (8.67%)
#Non-Fraudulent: 274 (91.33%) 
#Merchant_Risk counts: Low 188, Medium 84, High 28

💬 **Interpret this output:** What does it tell you about the data?

In [ ]:
#Distribution of amounts in transactions
sns.histplot(df["Amount"], bins=20)
plt.title("Distribution of amounts in transactions")
plt.show()

#This shows that most transactions are below 200, with a few outliers above that amount.
#At first glance it seems that fraudulent transactions are more likely to be of higher amounts, but further analysis is needed to confirm this.

💬 **Interpret this output:** What does it tell you about the data?

In [ ]:
# Boxplot of Amount by Merchant Risk
sns.boxplot(x="Merchant_Risk", y="Amount", data=df)
plt.title("Amount by Merchant Risk")
plt.xticks(rotation=90)
plt.show()

#High-risk merchants actually have a lower median and tighter spread of amounts compared to low and medium-risk merchants.

💬 **Interpret this output:** What does it tell you about the data?

In [ ]:
# Boxplot of Amount by Time of Day
sns.boxplot(x="Amount", y="Time_of_Day", data=df)
plt.title("Amount by Time of Day")
plt.show()

#Transactions in the afternoon (12-18) have a higher median amount and wider spread, while early morning (0-6) transactions are generally lower amounts.
#There isn't a clear link between time of day and fraud from this plot.

💬 **Interpret this output:** What does it tell you about the data?

In [ ]:
# Boxplot of Amount by Time of Day
sns.boxplot(x="Fraudulent", y="Amount", data=df)
plt.title("Amount by Fraudulent")
plt.show()

#Non-fraudulent transactions have a higher median amount and wider spread, indicating fraudsters may try to strear clear of higher-value transactions.

In [ ]:
#Create Amount bins
df["Amount_bin"] = pd.cut(df["Amount"],                     
bins=[0,50,100,200,500], 
labels=["0-50","50-100","100-200","200-500"])

#Group by Merchant_Risk and Amount_bin, compute fraud rate (%)
fraud_rate_combo = (
df.groupby(["Merchant_Risk", "Amount_bin"])["Fraudulent"]
.mean()
.reset_index()
)

fraud_rate_combo["Fraudulent"] = fraud_rate_combo["Fraudulent"] * 100

#Pivot table for heatmap
pivot_table = fraud_rate_combo.pivot(index="Merchant_Risk", 
columns="Amount_bin", 
values="Fraudulent")

# Plot heatmap
plt.figure(figsize=(8,5))
sns.heatmap(pivot_table, annot=True, fmt=".1f", cmap="Reds", cbar_kws={'label': 'Fraud Rate (%)'})
plt.title("Fraud Rate (%) by Merchant Risk and Amount Bin")
plt.ylabel("Merchant Risk")
plt.xlabel("Transaction Amount Range")
plt.tight_layout()
plt.show()

#Fraud is most frequent in High-risk merchants (as expected), but surprisingly, Low-risk merchants still show a non-negligible rate.
#By amount, fraud peaks around 100–200 but actually drops for very high transactions (200+). This could mean fraudsters prefer medium-value transactions to avoid detection.